In [1]:
!pip install --quiet datasets wandb

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import AutoTokenizer, GPTNeoConfig, GPTNeoForCausalLM
import json
import matplotlib.pyplot as plt
from datasets import load_dataset
from torch.utils.data import DataLoader
import wandb

In [3]:
configdict = { 
    "gpt-20M": { 
        "batch_size": 32,
        "block_size": 256,
        "lr": 5e-4,
        "n_layer": 6,
        "n_head": 6,
        "n_embd": 288,
        "dropout": 0.1,
        "weight_decay": 0.01,
        "epochs": 1,
        "eval_interval": 200,
        "eval_steps": 50,
        "vocab_size": 50257, #8000
        "warmup_tokens": 10000,
        "gradient_accumulation_steps": 8,
    },
    "gpt-8M": {
        "batch_size": 64,
        "block_size": 128,
        "lr": 5e-4,
        "n_layer": 8,
        "n_head": 8,
        "n_embd": 128,
        "dropout": 0.1,
        "weight_decay": 0.01,
        "epochs": 1,
        "eval_interval": 200,
        "eval_steps": 50,
        "vocab_size": 50257, #8000
        "warmup_tokens": 10000,
        "gradient_accumulation_steps": 16,
    },
    "gpt-3M": {
        "batch_size": 64,
        "block_size": 128,
        "lr": 6e-4,
        "n_layer": 4,
        "n_head": 4,
        "n_embd": 64,
        "dropout": 0.1,
        "weight_decay": 0.01,
        "epochs": 1,
        "eval_interval": 200,
        "eval_steps": 50,
        "vocab_size": 50257,
        "warmup_tokens": 5000,
        "gradient_accumulation_steps": 16,
    },
    "tokenizer": {
        "name": "EleutherAI/gpt-neo-125M",
    },
    "data": {
        "name": "roneneldan/TinyStories",
    },
}

In [4]:
class Config:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            if isinstance(value, dict):
                setattr(self, key, Config(value))
            else:
                setattr(self, key, value)

    def __getitem__(self, key):
        return self.__dict__[key]

config = Config(configdict)

In [5]:
torch.cuda.empty_cache()

In [6]:
###If running on Colab
#import os

#my_secret = os.getenv("WANDB_API_KEY")

#wandb_config = {k: v for k, v in vars(config).items() if not callable(getattr(config, k)) and not k.startswith("__")}  # Creating Wandb hyperparameters config for tracking experiments

#wandb.login(key=my_secret)
#run = wandb.init(project="gptneo-tinystories", name="tinystories-8M-3", config=wandb_config)
#wandb_config

In [7]:
from kaggle_secrets import UserSecretsClient

secret_label = "WANDB_API_KEY"
my_secret = UserSecretsClient().get_secret(secret_label)

wandb_config = {k: v for k, v in vars(config).items() if not callable(getattr(config, k)) and not k.startswith("__")}  # Creating Wandb hyperparameters config for tracking experiments

wandb.login(key=my_secret)
run = wandb.init(project="gptneo-tinystories", name="tinystories20M-5", config=wandb_config)
wandb_config

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anirudhr1201. Use `wandb login --relogin` to force relogin


{'gpt-20M': <__main__.Config at 0x7e5256dcb310>,
 'gpt-8M': <__main__.Config at 0x7e5256dcb670>,
 'gpt-3M': <__main__.Config at 0x7e5256dcb700>,
 'tokenizer': <__main__.Config at 0x7e5256dcb6d0>,
 'data': <__main__.Config at 0x7e5256dcb2e0>}

In [8]:
class TokenizerOLD:
    def __init__(self, config, device="cuda"):
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(config.name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.vocab_size = self.tokenizer.vocab_size

    def encoder(self, input, padding=True, max_length=256, truncation=True):
        #return self.tokenizer(input, return_tensors='pt', padding=padding, max_length=max_length, truncation=truncation)['input_ids'].to(self.device)
        encoded = self.tokenizer(input, return_tensors='pt', padding=padding, max_length=max_length, truncation=truncation)
        return {
            'input_ids': encoded['input_ids'].to(self.device),
            'attention_mask': encoded['attention_mask'].to(self.device)
        }
    def decoder(self, tokens):
        return self.tokenizer.decode(tokens[0], skip_special_tokens=True)


In [9]:
class Tokenizer:
    def __init__(self, config, k=None, file_path="None", device="cuda"):
        self.k = k
        self.file_path = file_path
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(config.name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.vocab_size = self.tokenizer.vocab_size if not self.k else self.k
        self.initialize()

    def get_config(self):
        config = {
            "initl_vocab_size": self.tokenizer.vocab_size,
            "final_vocab_size": self.vocab_size,
            "vocab_size": self.vocab_size,
            "total_tokens": self.total_tokens,
            "total_tokens_used": self.tokens_used if self.k else self.total_tokens,
            "total_unsed_tokens": self.total_tokens - self.tokens_used if self.k else 0
        }
        return config

    def initialize(self):
        with open(self.file_path, 'r') as file:
            tokens_counts = json.load(file)

        self.total_tokens = sum(tokens_counts.values())  # Already sorted

        if self.k:
            self.tokens_used = sum([i for i in tokens_counts.values()][:self.k])
            self.top_k_tokens = [i for i in tokens_counts.keys()][:self.k]  # We will only use top k tokens, others will be ignored
            self.top_k_tokens.append("50256")
            self.vocab_size += 1
            self.top_k_tokens_dict = {token: index for index, token in enumerate(self.top_k_tokens)}
            self.reversed_top_k_tokens_dict = {value: int(key) for key, value in self.top_k_tokens_dict.items()}

    def encoder(self, input, padding=False, max_length=512, truncation=False):
        tokens = self.tokenizer(input, return_tensors='pt', padding=padding, max_length=max_length, truncation=truncation)['input_ids'].to(self.device)

        if self.k:
            tokens = torch.tensor([self.top_k_tokens_dict.get(str(token.item()), self.top_k_tokens_dict["50256"]) for token in tokens.view(-1)], device=self.device).view(tokens.shape)

        return tokens
    
    def decoder(self, tokens):
        if self.k:
            tokens = torch.tensor([[self.reversed_top_k_tokens_dict.get(token.item(), self.reversed_top_k_tokens_dict[self.vocab_size - 1]) for token in row] for row in tokens], device=tokens.device)

        output = [self.tokenizer.decode(x, skip_special_tokens=True) for x in tokens]

        return output

In [10]:
class GPTModel(nn.Module):
    def __init__(self, config, device='cuda'):
        super().__init__()
        self.device = device
        self.block_size = config.block_size

        self.gpt_neo_config = GPTNeoConfig(
            vocab_size=config.vocab_size,
            max_position_embeddings=config.block_size,
            hidden_size=config.n_embd,
            num_layers=config.n_layer,
            num_heads=config.n_head,
            attention_types=[[["global", "local"], config.n_layer//2]],
            intermediate_size=config.n_embd * 4,
            activation_function="gelu_new",
            resid_dropout=config.dropout,
            embed_dropout=config.dropout,
            attention_dropout=config.dropout,
            layer_norm_epsilon=1e-5,
            initializer_range=0.02,
            use_cache=True,
            pad_token_id=50256,  
            eos_token_id=50256
        )

        self.model = GPTNeoForCausalLM(self.gpt_neo_config).to(device)
    
    def get_config(self):
        return self.gpt_neo_config

    def get_parameters(self):
        return sum(p.numel() for p in self.parameters())

    def save(self, path):
        torch.save(self.model.state_dict(), path)

    def forward(self, idx, targets=None):
        try:
            outputs = self.model(idx, labels=targets)
            return outputs.logits, outputs.loss
        except RuntimeError as e:
            print(f"Error in forward pass: {str(e)}")
            print(f"Input shape: {idx.shape}")
            print(f"Target shape: {targets.shape if targets is not None else 'None'}")
            raise

    def generate(self, idx, max_tokens, temperature=0.3, top_k=None):
        gen_kwargs = {
            "max_length": idx.shape[1] + max_tokens,
            "temperature": temperature,
            "top_k": top_k,
            "do_sample": True,
            "pad_token_id": self.model.config.pad_token_id,
        }

        generated = self.model.generate(idx, **gen_kwargs)
        return generated

In [11]:
def load_data(config, batch_size, device='cuda'):
    dataset = load_dataset(config.name)
    generator = torch.Generator(device=device)
    generator.manual_seed(42)
    train_data = DataLoader(dataset["train"]["text"], batch_size=batch_size, shuffle=True, pin_memory=True, generator=generator)
    val_data = DataLoader(dataset["validation"]["text"], batch_size=batch_size, shuffle=True, pin_memory=True, generator=generator)
    return train_data, val_data

In [12]:
@torch.no_grad()
def estimate_loss(model, train_data, val_data, encoder, eval_steps=50):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_steps)
        for k in range(eval_steps):
            data = train_data if split == 'train' else val_data
            tokens = encoder(next(iter(data)), max_length=model.block_size, padding="max_length", truncation=True)
            _, loss = model(tokens, tokens)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [13]:
class Trainer:
    def __init__(self, config, model, optimizer, train_data, val_data, encoder, scaler, device='cuda', scheduler=None):
        self.config = config
        self.model = model.to(device)
        self.optimizer = optimizer
        self.train_data = train_data
        self.val_data = val_data
        self.encoder = encoder
        self.scheduler = scheduler
        self.scaler = torch.cuda.amp.GradScaler() if device == 'cuda' else scaler
        self.device = device
        self.best_val_loss = float('inf')
        self.latest_best_model = ""
        self.gradient_accumulation_steps = config.gradient_accumulation_steps

    def trainOLD(self, epochs, eval_interval=200, eval_steps=50):
        for epoch in range(epochs):
            self.model.train()
            for i, batch in enumerate(self.train_data):
                tokens = self.encoder(batch, max_length=self.config.block_size, padding="max_length", truncation=True)
                _, loss = self.model(tokens, tokens)
                print(f"Epoch: {epoch + 1}/{epochs} | Batch: {i + 1}/{len(self.train_data)} | Loss: {loss.item():.4f}")

                # Normalize the loss to account for gradient accumulation
                loss = loss / self.gradient_accumulation_steps
                loss.backward()

                if (i + 1) % self.gradient_accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                    self.optimizer.step()
                    self.optimizer.zero_grad()
                    if self.scheduler:
                        self.scheduler.step()

            if (epoch + 1) % eval_interval == 0 or epoch == epochs - 1:
                losses = estimate_loss(self.model, self.train_data, self.val_data, self.encoder, eval_steps)
                print(f"Epoch: {epoch + 1}/{epochs} | Train loss: {losses['train']:.4f} | Val loss: {losses['val']:.4f}")
                wandb.log({
                    "epoch": epoch,
                    "train/loss": losses['train'],
                    "val/loss": losses['val'],
                    "lr": self.optimizer.param_groups[0]['lr']
                })
                if losses['val'] < self.best_val_loss:
                    self.best_val_loss = losses['val']
                    torch.save(self.model.state_dict(), f"checkpoint_epoch_{epoch+1}.pt")
                    self.latest_best_model = f"checkpoint_epoch_{epoch+1}.pt"

        return self.latest_best_model

    def train(self, epochs, eval_interval=200, eval_steps=50, generator=None):
        max_steps = epochs * len(self.train_data)
        steps = 0
        tracked_losses = []

        for epoch in range(epochs):
            self.model.train()
            for batch_idx, batch in enumerate(self.train_data):
                tokens = self.encoder(batch, max_length=self.config.block_size, padding="max_length", truncation=True)
                #tokens = {k: v.to(self.device) for k, v in tokens.items()}#tokens.to(self.device)

                with torch.amp.autocast('cuda', dtype=torch.float16):
                    _, output_loss = self.model(tokens, tokens)
                    loss = output_loss / self.gradient_accumulation_steps

                self.scaler.scale(loss).backward()

                if (batch_idx + 1) % self.gradient_accumulation_steps == 0 or (batch_idx + 1) == len(self.train_data):
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                    self.optimizer.zero_grad(set_to_none=True)
                    if self.scheduler:
                        self.scheduler.step()
                    steps += 1

                    if steps % eval_interval == 0 or steps == max_steps-1:
                        print(f"Starting Epoch: {epoch + 1} {'-' * 100}")
                        losses = estimate_loss(self.model, self.train_data, self.val_data, self.encoder, eval_steps)
                        tracked_losses.append(losses)
                        print(f"Epoch: {epoch + 1}/{epochs} | Step: {steps} | Train loss: {losses['train']:.4f} | Val loss: {losses['val']:.4f}")
                        wandb.log({
                            "epoch": epoch,
                            "train/loss": losses['train'],
                            "val/loss": losses['val'],
                            "lr": self.optimizer.param_groups[0]['lr']
                        })
                        if losses['val'] < self.best_val_loss:
                            self.best_val_loss = losses['val']
                            torch.save(self.model.state_dict(), f"checkpoint_epoch_{epoch+1}.pt")
                            self.latest_best_model = f"checkpoint_epoch_{epoch+1}.pt"

                        else:
                            print("Validation loss did not improve, consider stopping early.")

        return self.latest_best_model


In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)

model_name = "gpt-20M"
model_config = config[model_name]

In [15]:
tokenizer = Tokenizer(config.tokenizer, file_path="/kaggle/input/tokengptneo/tokens.json", device=device)
train_data, val_data = load_data(config.data, model_config.batch_size, device=device)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

...

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

In [16]:
model = GPTModel(model_config, device=device)
print(f"Model has {model.get_parameters():,} parameters")
model = model.to(device)

Model has 3,423,936 parameters


In [17]:
optim = torch.optim.AdamW(model.parameters(), lr=model_config.lr, weight_decay=model_config.weight_decay)
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
scaler = torch.amp.GradScaler('cuda', enabled=(dtype == 'float16'))

In [18]:
trainer = Trainer(model_config, model, optim, train_data, val_data, tokenizer.encoder, scaler, device)
bestmodel = trainer.train(epochs=model_config.epochs, eval_interval=model_config.eval_interval, eval_steps=model_config.eval_steps)

/tmp/ipykernel_30/4227591036.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler() if device == 'cuda' else scaler


Starting Epoch: 1 ----------------------------------------------------------------------------------------------------
Epoch: 1/1 | Step: 200 | Train loss: 5.1727 | Val loss: 5.1601
Starting Epoch: 1 ----------------------------------------------------------------------------------------------------
Epoch: 1/1 | Step: 400 | Train loss: 3.9857 | Val loss: 3.9825
Starting Epoch: 1 ----------------------------------------------------------------------------------------------------
Epoch: 1/1 | Step: 600 | Train loss: 3.5977 | Val loss: 3.5710


KeyboardInterrupt: 

In [20]:
#bestmodel = f"checkpoint_epoch_1.pt"
checkpoint = torch.load(bestmodel)
model.load_state_dict(checkpoint) 
model.eval()

/tmp/ipykernel_30/1746287869.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(bestmodel)


GPTModel(
  (model): GPTNeoForCausalLM(
    (transformer): GPTNeoModel(
      (wte): Embedding(50257, 64)
      (wpe): Embedding(128, 64)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-3): 4 x GPTNeoBlock(
          (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (attn): GPTNeoAttention(
            (attention): GPTNeoSelfAttention(
              (attn_dropout): Dropout(p=0.1, inplace=False)
              (resid_dropout): Dropout(p=0.1, inplace=False)
              (k_proj): Linear(in_features=64, out_features=64, bias=False)
              (v_proj): Linear(in_features=64, out_features=64, bias=False)
              (q_proj): Linear(in_features=64, out_features=64, bias=False)
              (out_proj): Linear(in_features=64, out_features=64, bias=True)
            )
          )
          (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (mlp): GPTNeoMLP(
            (c_fc): Linear(in_features=64, out_features=256,

In [ ]:
#torch.save(model.state_dict(), 'model.bin')
#model.eval()

In [21]:
import re

def load_model(config, path, device='cuda'):
    model = GPTModel(config, device=device)
    model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    model.to(device)
    model.eval()
    return model

def clean_string(input_string):
    print(input_string[0])
    cleaned_string = re.sub(r'[^\w\s.,]', '', input_string[0])
    cleaned_string = cleaned_string.replace('\n', '')
    return cleaned_string

In [22]:
#prompt ="'Hi Jane, have you seen Alice? I can't find her anywhere', said Jack"
#outputs = {}
#model.eval()
#for t in range(1,11):
#    for k in range(15):
#        if k==0:
#            output = model.generate(tokenizer.encoder(prompt), max_tokens=150, temperature=float(t/10), top_k=None)
#        else:
#            output = model.generate(tokenizer.encoder(prompt), max_tokens=150, temperature=float(t/10), top_k=k)
        #try:
#        outputs[(t/10,k)] = clean_string(tokenizer.decoder(output))
        #except:
        #outputs[(t/10,k)] = (tokenizer.decoder(output2))
#print(outputs)

In [23]:
prompt ="'Hi Jane, have you seen Alice? I can't find her anywhere', said Jack"
output = model.generate(tokenizer.encoder(prompt), max_tokens=100, temperature=0.2, top_k=None)
print(clean_string(tokenizer.decoder(output)))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'Hi Jane, have you seen Alice? I can't find her anywhere', said Jack.
One day, Lily's mommy's mommy's mommy's mommy and said, "I can go to the park. I have to the park!"

Lily said, "I'm very happy. "I want to play with the park. I have to go to the park. I want to the park. "I want to play with you to play with you to play with you!"

"Look, I can play with the park.


Hi Jane, have you seen Alice I cant find her anywhere, said Jack.One day, Lilys mommys mommys mommys mommy and said, I can go to the park. I have to the parkLily said, Im very happy. I want to play with the park. I have to go to the park. I want to the park. I want to play with you to play with you to play with youLook, I can play with the park.


In [ ]:
#import os
#from huggingface_hub import HfApi, Repository
#from kaggle_secrets import UserSecretsClient

#def save_to_huggingface(model, config, tokenizer, repo_name, commit_message="Update model"):
#    temp_dir = "/kaggle/working/"
#    os.makedirs(temp_dir, exist_ok=True)

#    model_path = os.path.join("/", "model.bin")
#    torch.save(model.state_dict(), 'pytorch_model.bin') 
    
#    config_dict = config.to_dict()
#    config_path = os.path.join(temp_dir, "config.json")
#    with open(config_path, 'w') as f:
#        print (config)
#        json.dump(config_dict, f, indent=2)

#    #tokenizer.tokenizer.save_pretrained(temp_dir)

#    # Get Hugging Face token from Kaggle secrets
#    user_secrets = UserSecretsClient()
#    hf_token = user_secrets.get_secret("huggingface_token")

#    # Initialize HfApi with the token
#    api = HfApi(token=hf_token)
    
#    api.upload_file(
#        path_or_fileobj="pytorch_model.bin",
#        path_in_repo="pytorch_model.bin",
#        repo_id=repo_id,
#        repo_type="model",
#    )
    

#    api.upload_file(
#        path_or_fileobj="config.json",
#        path_in_repo="config.json",
#        repo_id=repo_id,
#    )
    
#    api.upload_file(
#        path_or_fileobj="/kaggle/input/tokengptneo/tokens.json",
#        path_in_repo="tokens.json",
#        repo_id=repo_id,
#    )
    


#    #repo_url = api.create_repo(repo_name, exist_ok=True)
#    #repo = Repository(local_dir="/", clone_from=repo_url)

#    #repo.git_add()
#    #repo.git_commit(commit_message)
#    #repo.git_push()

#    #print(f"Model, config, and tokenizer saved to: {repo_url}")

#    #for file in os.listdir(temp_dir):
#    #    os.remove(os.path.join(temp_dir, file))
#    #os.rmdir(temp_dir)

    
    
#repo_id = "AnirudhRajagopalan1201/tinystories-custom-3M"
#model_config = model.get_config()
#model_config.save_pretrained('./')
#save_to_huggingface(model, model_config, None, repo_id)


In [25]:
from huggingface_hub import notebook_login, login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("huggingface_token")
login(token=hf_token)
repo ="AnirudhRajagopalan1201/tinystories-custom-20M" 
model.model.save_pretrained("./")
model.model.push_to_hub(repo)
tokenizer.tokenizer.save_pretrained("./")
tokenizer.tokenizer.push_to_hub(repo)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


model.safetensors:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AnirudhRajagopalan1201/tinystories-custom-3M/commit/c5dfcbfb523ecbfdbf8cec9dbaeede806698fc36', commit_message='Upload tokenizer', commit_description='', oid='c5dfcbfb523ecbfdbf8cec9dbaeede806698fc36', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AnirudhRajagopalan1201/tinystories-custom-3M', endpoint='https://huggingface.co', repo_type='model', repo_id='AnirudhRajagopalan1201/tinystories-custom-3M'), pr_revision=None, pr_num=None)